In [ ]:
import pandas as pd
import geopandas as gpd
from pandas_geojson import to_geojson
from shapely.geometry import Point
import json


In [17]:
#Create path and red in CSV
path = 'Wildfires_2.csv'
csv = pd.read_csv(path)
#Convert CSV into DF
df = pd.DataFrame(csv)

In [18]:
first_years_df = df[df["FIRE_YEAR"].between(1992, 1999)]
second_years_df = df[df["FIRE_YEAR"].between(2000, 2007)]
third_years_df = df[df["FIRE_YEAR"].between(2008, 2015)]

In [19]:
#Store points for coordinates in a variable
geometry = [Point(xy) for xy in zip(first_years_df['LONGITUDE'], first_years_df['LATITUDE'])]
#Create GeoDataFrame and add geometry column
geo_first_years = gpd.GeoDataFrame(first_years_df, geometry=geometry)

#Sort Values
gdf_first_years = gdf_first_years.sort_values('FIRE_YEAR')

#Drop unnecessary columns
gdf_first_years = geo_first_years.drop(['LATITUDE', 'LONGITUDE', 'index', 'STAT_CAUSE_DESCR', 'Fire_Date'], axis = 1)
gdf_first_years = gdf_first_years.dropna()

#Convert non-spatial columns to properties
properties = gdf_first_years.drop(['geometry'], axis = 1).to_dict(orient='records')

# Convert Point geometries to GeoJSON-compatible format
gdf_first_years['geometry'] = gdf_first_years['geometry'].apply(lambda point: {'coordinates': (point.x, point.y)})

#Add properties to GeoJSON
geojson_data_first_years = {
    "features": []
}

for feature, prop in zip(gdf_first_years['geometry'], properties):
    geojson_data_first_years['features'].append({
        "geometry": feature,
        "properties": prop
    })
    
# Save GeoJSON to a file
with open('first_years.geojson', 'w') as f:
    json.dump(geojson_data_first_years, f, indent=2)

d:\Anaconda\envs\dev\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [20]:
#Store points for coordinates in a variable
geometry = [Point(xy) for xy in zip(second_years_df['LONGITUDE'], second_years_df['LATITUDE'])]

#Create GeoDataFrame and add geometry column
geo_second_years = gpd.GeoDataFrame(second_years_df, geometry=geometry)

#Sort Values
geo_second_years = geo_second_years.sort_values('FIRE_YEAR')

#Drop unnecessary columns
gdf_second_years = geo_second_years.drop(['LATITUDE', 'LONGITUDE', 'index', 'STAT_CAUSE_DESCR', 'Fire_Date'], axis = 1)
gdf_second_years = gdf_second_years.dropna()

#Convert non-spatial columns to properties
properties = gdf_second_years.drop(['geometry'], axis = 1).to_dict(orient='records')

# Convert Point geometries to GeoJSON-compatible format
gdf_second_years['geometry'] = gdf_second_years['geometry'].apply(lambda point: {'coordinates': (point.x, point.y)})

#Add properties to GeoJSON
geojson_data_second_years = {
    "features": []
}

for feature, prop in zip(gdf_second_years['geometry'], properties):
    geojson_data_second_years['features'].append({
        "geometry": feature,
        "properties": prop
    })
    
# Save GeoJSON to a file
with open('second_years.geojson', 'w') as f:
    json.dump(geojson_data_second_years, f, indent=2)

d:\Anaconda\envs\dev\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [21]:
#Store points for coordinates in a variable
geometry = [Point(xy) for xy in zip(third_years_df['LONGITUDE'], third_years_df['LATITUDE'])]
#Create GeoDataFrame and add geometry column
geo_third_years = gpd.GeoDataFrame(third_years_df, geometry=geometry)

#Sort Values
geo_third_years = geo_third_years.sort_values('FIRE_YEAR')

#Drop unnecessary columns
gdf_third_years = geo_third_years.drop(['LATITUDE', 'LONGITUDE', 'index', 'STAT_CAUSE_DESCR', 'Fire_Date'], axis = 1)
gdf_third_years = gdf_third_years.dropna()

#Convert non-spatial columns to properties
properties = gdf_third_years.drop(['geometry'], axis = 1).to_dict(orient='records')

# Convert Point geometries to GeoJSON-compatible format
gdf_third_years['geometry'] = gdf_third_years['geometry'].apply(lambda point: {'coordinates': (point.x, point.y)})

#Add properties to GeoJSON
geojson_data_third_years = {
    "features": []
}

for feature, prop in zip(gdf_third_years['geometry'], properties):
    geojson_data_third_years['features'].append({
        "geometry": feature,
        "properties": prop
    })
    
# Save GeoJSON to a file
with open('third_years.geojson', 'w') as f:
    json.dump(geojson_data_third_years, f, indent=2)

d:\Anaconda\envs\dev\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
